<a href="https://colab.research.google.com/github/liligaohui/SIAS_Global-Equity-Risk-Performance-Analysis/blob/main/Global_Risk_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SIAS Risk Performance Analysis

Code by Lili Gao

Last updated: July 31, 2025


## Content of the Anlysis

**1. Portfolio Metrics Based on Current Weights**

* **Active Weight (Individual Securities):**
  Quantifies the difference in each position’s weight between the SIAS portfolio and its benchmark, highlighting individual over- or under-allocations.
* **Active Sector Exposure:**
  Measures sector-level over- or under-weights relative to the benchmark, revealing sector-tilt decisions.
* **Position Concentration:**
  Assessed via the Herfindahl–Hirschman Index (HHI), top-10 holdings weight, etc., to capture single-name risk and overall diversification.

---

**2. Metrics Requiring Historical Price/Return Data**

* **Volatility:**
  Standard deviation of portfolio returns.
* **Beta (vs. ACWI):**
  Sensitivity of portfolio returns to the MSCI ACWI index.
* **Tracking Error:**
  Standard deviation of the portfolio’s active return (portfolio minus benchmark).
* **Information Ratio:**
  Ratio of active return to tracking error, measuring risk-adjusted outperformance.
* **Sharpe & Sortino Ratios:**
  Reward-to-risk metrics based on total downside or downside-only volatility.
* **Maximum Drawdown:**
  Largest peak-to-trough decline in portfolio value.
* **Value-at-Risk (VaR) / Expected Shortfall (ES):**
  Quantify potential downside at specified confidence levels.

---

**3. Metrics from Factor or Risk Models**

* **Marginal & Component VaR:**
  Breaks down total VaR by individual names or risk factors to pinpoint primary risk drivers.
* **Stress-Test Profit & Loss:**
  Simulate predefined shocks (e.g.,–10 % in tech sectors) to estimate portfolio P/L under adverse scenarios.

---

**4. Combined Performance-and-Holdings Diagnostics**

* **Partial Performance Attribution:**
  Decomposes active return into allocation and selection effects across sectors.
* **Hit Ratio (Batting Average):**
  Percentage of periods (daily, monthly) in which the portfolio outperforms its benchmark.
* **Upside / Downside Capture Ratios:**
  Measure performance in up-market versus down-market environments, indicating resilience and downside protection.

## Download Data

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Colab path
import os

folder = '/content/drive/MyDrive/Colab Notebooks/SIAS_Risk Monitoring'
print(os.listdir(folder))

# SIAS Portoflio File download
filename = 'SIAS Asset Detail_25 Jul 2025.xlsx'  # <- adjust if it’s different
file_path = os.path.join(folder, filename)

# Benchmark File download
folder2    = '/content/drive/MyDrive/Colab Notebooks/SIAS_Risk Monitoring'
filename2  = 'ACWI_holdings_29 July 2025.csv'
file_path2 = os.path.join(folder2, filename2)

['ACWI_holdings_29 July 2025.csv', 'SIAS Asset Detail_25 Jul 2025.xlsx', 'Share_portion.ipynb']


## Portfolio Metrics Based on Current Weights

### Benchmark Overall Performance (iShares MSCI ACWI ETF)

In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# 1) Read the CSV, skipping initial metadata rows
df_bm = pd.read_csv(file_path2, skiprows=9)

# 2) Select only Ticker, Name, Sector Weight, and Price
#    (adjust these names to exactly match your CSV’s headers)
df_bm_sel = df_bm[['Ticker', 'Name','Sector', 'Price', 'Weight (%)']]

# 3) Sorting
metrics_bm = ['Ticker', 'Name','Sector', 'Price', 'Weight (%)']
metric_dd_bm = widgets.Dropdown(options=metrics_bm, value='Weight (%)', description='Sort by:')
order_tb_bm = widgets.ToggleButtons(options=['Descending','Ascending'], description='Order:')
out_bm = widgets.Output()

def update_bm(sort_by, order):
    asc = (order == 'Ascending')
    df_sorted = df_bm_sel.sort_values(by=sort_by, ascending=asc)
    with out_bm:
        clear_output(wait=True)
        display(df_sorted)

# 4) Print
# update_bm(metric_dd_bm.value, order_tb_bm.value)
# widgets.interact(update_bm, sort_by=metric_dd_bm, order=order_tb_bm)
# display(widgets.HBox([metric_dd_bm, order_tb_bm]), out_bm)

### SIAS Portfolio Overall Performance (Global Market)

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display, clear_output

# 1) Read & filter
df = pd.read_excel(file_path, engine='openpyxl')
# Explicitly keep 'Security Description 1' column here

# Rename the column once
df = df.rename(columns={'Security Description 1': 'Name'})

# Filter to non-Canadian rows and keep the desired columns
df_usd = df.loc[
    df['Country Name'].str.upper() != 'CANADA',
    ['Ticker', 'Name', 'Shares/Par', 'Country Name']
].copy()

df_usd = df_usd[df_usd['Ticker'].notna()]
df_usd['Ticker'] = df_usd['Ticker'].str.strip().str.upper()

# 2) Bulk download history - Using a loop to fetch data for each ticker
def fetch_price_and_log_returns(ticker: str):
    """
    Returns: (latest_price, log_ret_1d, log_ret_1w, log_ret_1m)
    - 1d  = ln(P_t / P_{t-1})
    - 1w  = ln(P_t / P_{t-5})
    - 1m  = ln(P_t / P_{t-21})
    """
    hist = (
        yf.Ticker(ticker.upper())
          .history(period='2mo', interval='1d')['Close']
          .dropna()
    )
    if len(hist) < 2:
        return (np.nan, np.nan, np.nan, np.nan)

    latest = hist.iloc[-1]

    # 1-day
    ret1d = np.log(latest / hist.iloc[-2])

    # ~5 trading days (1-week)
    ret1w = np.log(latest / hist.asof(hist.index[-6])) if len(hist) >= 6 else np.nan

    # ~21 trading days (1-month)
    ret1m = np.log(latest / hist.asof(hist.index[-22])) if len(hist) >= 22 else np.nan

    return latest, ret1d, ret1w, ret1m

# Apply and unpack
cols = ['Price_USD', 'LogRet_1D', 'LogRet_1W', 'LogRet_1M']
df_usd[cols] = df_usd['Ticker'].apply(fetch_price_and_log_returns).tolist()


# 4) Market values & weights
df_usd['Market_Value_USD'] = df_usd['Shares/Par'] * df_usd['Price_USD']
total = df_usd['Market_Value_USD'].sum()
df_usd['Portfolio_Weight_%'] = df_usd['Market_Value_USD'] / total * 100

# 5) Interactive table
metrics = ['Price_USD','LogRet_1D','LogRet_1W','LogRet_1M','Market_Value_USD','Portfolio_Weight_%']
metric_dd = widgets.Dropdown(options=metrics, value='Portfolio_Weight_%', description='Sort by:')
order_tb  = widgets.ToggleButtons(options=['Descending','Ascending'], description='Order:')
out       = widgets.Output()

def update(sort_by, order):
    ascending = (order == 'Ascending')
    df_sorted = df_usd.sort_values(by=sort_by, ascending=ascending)
    with out:
        clear_output(wait=True)
        display(
            df_sorted[
                ['Ticker', 'Name', 'Shares/Par', 'Country Name']  # ← added Name here
                + metrics
            ]
        )

# show initial and wire up
update(metric_dd.value, order_tb.value)
widgets.interact(update, sort_by=metric_dd, order=order_tb)
display(metric_dd, order_tb, out)

interactive(children=(Dropdown(description='Sort by:', index=5, options=('Price_USD', 'LogRet_1D', 'LogRet_1W'…

Dropdown(description='Sort by:', index=5, options=('Price_USD', 'LogRet_1D', 'LogRet_1W', 'LogRet_1M', 'Market…

ToggleButtons(description='Order:', options=('Descending', 'Ascending'), value='Descending')

Output()

### Active Performance - Active Individuals' Weight

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# ───────────────────────────────────────────────────────────────
# 0)  PREP BENCHMARK TABLE
# ───────────────────────────────────────────────────────────────
df_bm_sel = (
    df_bm
      .rename(columns={'Weight (%)': 'Benchmark_Weight_%'})
      .loc[:, ['Ticker', 'Name', 'Sector', 'Price', 'Benchmark_Weight_%']]
)

df_bm_sel['Ticker'] = df_bm_sel['Ticker'].astype(str).str.strip().str.upper()
df_usd['Ticker']    = df_usd['Ticker'].astype(str).str.strip().str.upper()

# Handle duplicate tickers in the benchmark data by keeping the first occurrence
df_bm_sel = df_bm_sel.drop_duplicates(subset=['Ticker'], keep='first')


# ───────────────────────────────────────────────────────────────
# 1)  FIRST MERGE — by Ticker
# ───────────────────────────────────────────────────────────────
df_combined = (
    df_usd[['Ticker', 'Name', 'Shares/Par', 'Country Name',
            'Price_USD', 'Market_Value_USD', 'Portfolio_Weight_%']]
    .merge(df_bm_sel[['Ticker', 'Sector', 'Benchmark_Weight_%']],
           on='Ticker', how='left', suffixes=('', '_bm'))
)

# ───────────────────────────────────────────────────────────────
# 2)  SECOND PASS — fuzzy match on Name
# ───────────────────────────────────────────────────────────────
mask_missing = df_combined['Benchmark_Weight_%'].isna()

if mask_missing.any():
    # helper: strip apostrophes, take first n words, lowercase
    def first_n_words(txt, n):
        if pd.isna(txt):
            return ''
        clean = str(txt).replace("'", '').strip().lower()   # ← apostrophes removed
        return ' '.join(clean.split()[:n])

    bm_tmp = df_bm_sel.dropna(subset=['Name']).copy()
    bm_tmp['key2'] = bm_tmp['Name'].apply(lambda x: first_n_words(x, 2))
    bm_tmp['key1'] = bm_tmp['Name'].apply(lambda x: first_n_words(x, 1))

    look2 = (
        bm_tmp.drop_duplicates(subset='key2', keep='first')
              .set_index('key2')[['Sector', 'Benchmark_Weight_%']]
              .to_dict(orient='index')
    )
    look1 = (
        bm_tmp.drop_duplicates(subset='key1', keep='first')
              .set_index('key1')[['Sector', 'Benchmark_Weight_%']]
              .to_dict(orient='index')
    )

    def _fill_by_name(row):
        if not pd.isna(row['Benchmark_Weight_%']):
            return row
        key2 = first_n_words(row['Name'], 2)
        hit  = look2.get(key2) if key2 else None
        if not hit:
            key1 = first_n_words(row['Name'], 1)
            hit  = look1.get(key1) if key1 else None
        if hit:
            row['Sector']             = hit['Sector']
            row['Benchmark_Weight_%'] = hit['Benchmark_Weight_%']
        return row

    df_combined.loc[mask_missing] = (
        df_combined.loc[mask_missing]
                   .apply(_fill_by_name, axis=1)
    )

# ───────────────────────────────────────────────────────────────
# 3)  ACTIVE WEIGHT
# ───────────────────────────────────────────────────────────────
df_combined['Benchmark_Weight_%'] = df_combined['Benchmark_Weight_%'].fillna(0.0)
df_combined['Active_Weight_%']    = (
    df_combined['Portfolio_Weight_%'] - df_combined['Benchmark_Weight_%']
)

# ───────────────────────────────────────────────────────────────
# 4)  INTERACTIVE SORT UI
# ───────────────────────────────────────────────────────────────
sort_columns = [
    'Portfolio_Weight_%', 'Benchmark_Weight_%', 'Active_Weight_%',
    'Sector', 'Price_USD', 'Market_Value_USD'
]

sort_dd  = widgets.Dropdown(options=sort_columns,
                            value='Active_Weight_%',
                            description='Sort by:')
order_tb = widgets.ToggleButtons(options=['Descending', 'Ascending'],
                                 value='Descending',
                                 description='Order:')
out = widgets.Output()

def _update(sort_by, order):
    asc = (order == 'Ascending')
    with out:
        clear_output(wait=True)
        display(
            df_combined
              .sort_values(by=sort_by, ascending=asc)
              .reset_index(drop=True)
        )

_update(sort_dd.value, order_tb.value)
widgets.interact(_update, sort_by=sort_dd, order=order_tb)
display(widgets.HBox([sort_dd, order_tb]), out)

interactive(children=(Dropdown(description='Sort by:', index=2, options=('Portfolio_Weight_%', 'Benchmark_Weig…

Output()

### Active Performance - Active Sectors' Weight

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# ───────────────────────────────────────────────────────────────
# 0)  PREP benchmark table  &  normalise “ETF holdings”
# ───────────────────────────────────────────────────────────────
df_bm_sel = (
    df_bm.rename(columns={'Weight (%)': 'Benchmark_Weight_%'})
          .loc[:, ['Ticker', 'Name', 'Sector', 'Price', 'Benchmark_Weight_%']]
)

# Tag missing sector as ETF holdings
df_bm_sel['Sector'] = df_bm_sel['Sector'].fillna('ETF holdings')
df_combined['Sector'] = df_combined['Sector'].fillna('ETF holdings')

# Ensure numeric, NaNs → 0
df_combined['Portfolio_Weight_%'] = df_combined['Portfolio_Weight_%'].fillna(0.0)
df_bm_sel['Benchmark_Weight_%']   = df_bm_sel['Benchmark_Weight_%'].fillna(0.0)

# ───────────────────────────────────────────────────────────────
# 1)  PORTFOLIO sector weight
# ───────────────────────────────────────────────────────────────
port_sector = (
    df_combined.groupby('Sector', dropna=False)['Portfolio_Weight_%']
               .sum()
               .reset_index()
)

# ───────────────────────────────────────────────────────────────
# 2)  BENCHMARK sector weight  (all holdings)
# ───────────────────────────────────────────────────────────────
bm_sector = (
    df_bm_sel.groupby('Sector', dropna=False)['Benchmark_Weight_%']
             .sum()
             .reset_index()
)

# ───────────────────────────────────────────────────────────────
# 3)  MERGE & calculate active weight
# ───────────────────────────────────────────────────────────────
sector_tbl = (
    port_sector.merge(bm_sector, on='Sector', how='outer')
               .fillna(0.0)
)

sector_tbl['Active_Sector_%'] = (
    sector_tbl['Portfolio_Weight_%'] - sector_tbl['Benchmark_Weight_%']
)

# ───────────────────────────────────────────────────────────────
# 4)  INTERACTIVE sort widget
# ───────────────────────────────────────────────────────────────
sec_cols = ['Portfolio_Weight_%', 'Benchmark_Weight_%', 'Active_Sector_%']

sort_dd  = widgets.Dropdown(options=sec_cols,
                            value='Active_Sector_%',
                            description='Sort by:')
order_tb = widgets.ToggleButtons(options=['Descending', 'Ascending'],
                                 value='Descending',
                                 description='Order:')
sec_out  = widgets.Output()

def _update_sector(sort_by, order):
    asc = (order == 'Ascending')
    with sec_out:
        clear_output(wait=True)
        display(
            sector_tbl.sort_values(by=sort_by, ascending=asc)
                      .reset_index(drop=True)
        )

_update_sector(sort_dd.value, order_tb.value)
widgets.interact(_update_sector, sort_by=sort_dd, order=order_tb)
display(widgets.HBox([sort_dd, order_tb]), sec_out)


interactive(children=(Dropdown(description='Sort by:', index=2, options=('Portfolio_Weight_%', 'Benchmark_Weig…

Output()

## Diversification and Concentration Analysis

1. Herfindahl–Hirschman Index (HHI)

**Formula:**

$$
\text{HHI} \;=\; \sum_{i=1}^{M} w_i^2
$$

where $w_i$ is the **fractional** weight of holding $i$ in the portfolio (i.e. weight % divided by 100), and $M$ is the total number of holdings.

**Interpretation:**

* **Range:** $[0,1]$.
* **Low HHI** (close to 0) ⇒ highly diversified (many small, equal‐sized positions).
* **High HHI** (approaching 1) ⇒ dominated by one or few positions.
* **Example:** An equally weighted 100‐stock portfolio has HHI = 100 × (1/100)² = 0.01.

---

2. Effective Number of Holdings

**Formula:**

$$
\text{Effective N} \;=\; \frac{1}{\text{HHI}}
$$

**Interpretation:**

* Converts HHI into an “equivalent” count of **equal‐weighted** positions.
* **High Effective N** ⇒ behaves like a large equal‐weight portfolio (diversified).
* **Low Effective N** ⇒ concentration risk akin to a small portfolio, even if you hold many names.

---

3. Top-N Holdings Weight

**Formula:**

$$
\text{Top-N Weight} \;=\; \sum_{i \in \text{largest }N} w_i
$$

where you sort all holdings by descending weight $w_i$, then sum the top $N$ fractional weights. Multiply by 100 to express as a percentage.

**Interpretation:**

* Measures how much of your portfolio (or the benchmark) sits in its **largest $N$** names.
* A **high** Top-10 % (for example) indicates that the biggest 10 positions dominate your risk and return.
* Tuning $N$ (e.g. Top-5 vs. Top-20) reveals the “tail” of concentration.

In [ ]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# ------------------------------------------------------------------
# 0)  PREP weight Series -------------------------------------------------
# ------------------------------------------------------------------
w_port = (df_combined['Portfolio_Weight_%']  / 100).fillna(0.0)
w_bm   = (df_combined['Benchmark_Weight_%'] / 100).fillna(0.0)

# ------------------------------------------------------------------
# 1)  FUNCTIONS --------------------------------------------------------
# ------------------------------------------------------------------
def hhi(weights):
    """Herfindahl–Hirschman Index using fractional weights."""
    return np.square(weights).sum()

def effective_n(weights):
    h = hhi(weights)
    return np.inf if h == 0 else 1.0 / h

def top_n_weight(weights, n=10):
    return weights.sort_values(ascending=False).head(n).sum()

# ------------------------------------------------------------------
# 2)  MAIN CALC + WIDGET ------------------------------------------------
# ------------------------------------------------------------------
N_slider = widgets.IntSlider(value=10, min=1, max=30, step=1, description='Top-N:')

out = widgets.Output()

def _show_concentration(n):
    with out:
        clear_output(wait=True)
        metrics = {
            'Portfolio HHI'          : hhi(w_port),
            'Benchmark HHI'          : hhi(w_bm),
            'Effective # (Port)'     : effective_n(w_port),
            'Effective # (Bench)'    : effective_n(w_bm),
            f'Top-{n} % (Port)'      : top_n_weight(w_port, n)*100,
            f'Top-{n} % (Bench)'     : top_n_weight(w_bm, n)*100
        }
        df_metrics = (
            pd.Series(metrics, name='Value')
              .to_frame()
              .style.format('{:.4f}')
        )
        display(df_metrics)

_show_concentration(N_slider.value)
N_slider.observe(lambda change: _show_concentration(change['new']), names='value')
display(N_slider, out)


IntSlider(value=10, description='Top-N:', max=30, min=1)

Output()

## Risk Metrics

1. **Volatility (%)**

   * **Definition:** The annualized standard deviation of periodic returns, expressed as a percentage.
   * **Formula:**

     $$
     \text{Volatility} \=\\sigma_{\Delta p}\\sqrt{N}\\times100\%
     $$

     where $\sigma_{\Delta p}$ is the standard deviation of returns and $N$ is the number of periods per year (typically 252 trading days).
   * **Interpretation:** Measures the total variability of portfolio (or benchmark) returns—higher values indicate larger fluctuations (i.e., more “risk”).

2. **Sharpe Ratio**

   * **Definition:** The ratio of excess return to total risk.
   * **Formula:**

     $$
     \text{Sharpe} \=\\frac{\mu - r_f}{\sigma}\\sqrt{N}
     $$

     where $\mu$ is the average portfolio return, $r_f$ is the risk-free rate, $\sigma$ is the portfolio volatility, and $N$ is the number of periods per year.
   * **Interpretation:** Indicates how effectively the portfolio compensates for taking volatility risk—higher is better.

3. **Sortino Ratio**

   * **Definition:** The ratio of excess return to downside risk, penalizing only negative deviations.
   * **Formula:**

     $$
     \text{Sortino} \=\\frac{\mu - r_f}{\sigma_{\rm down}}\\sqrt{N}
     $$

     where $\sigma_{\rm down}$ is the standard deviation of negative returns.
   * **Interpretation:** Evaluates portfolio performance relative to downside volatility—focuses solely on downside risk.

4. **Maximum Drawdown (%)**

   * **Definition:** The largest peak-to-trough decline over the evaluation period, expressed as a percentage.
   * **Formula:**

     $$
     \max_{t}\\Bigl(\frac{P_t - \max_{s\le t}P_s}{\max_{s\le t}P_s}\Bigr)\times100\%
     $$

     where $P_t$ is the portfolio value at time $t$.
   * **Interpretation:** Reflects the worst cumulative loss experienced—critical for assessing drawdown risk.

5. **Value-at-Risk (VaR$_{95}$, %)**

   * **Definition:** The loss threshold such that 95% of daily returns exceed this bound (i.e., only 5% of losses are worse). Expressed as a negative percentage.
   * **Formula:**
     $$
     \ VaR_{95}
 =Percentile
5%
​
 (Δp)×100\%\
     $$
   * **Interpretation:** Provides a one-day “worst-loss” estimate with 95% confidence.

6. **Expected Shortfall (ES$_{95}$, %)**

   * **Definition:** The average loss given that returns breach the VaR$_{95}$ threshold.
   * **Formula:**

     $$
     ES_
{95}
​
 =E[Δp∣Δp≤VaR_
{95}
​
 ]×100%
     $$
   * **Interpretation:** Captures tail risk by quantifying the average loss in the worst 5% of outcomes.

7. **Beta vs. ACWI**

   * **Definition:** The sensitivity of portfolio returns to MSCI ACWI returns—measuring systematic market exposure.
   * **Formula:**

     $$
     \beta \=\\frac{\mathrm{Cov}(r_p,\,r_{\rm ACWI})}{\mathrm{Var}(r_{\rm ACWI})}
     $$
   * **Interpretation:** Indicates how closely the portfolio’s movements align with the global equity market.

8. **Tracking Error (%)**

   * **Definition:** The annualized standard deviation of active returns (portfolio minus benchmark).
   * **Formula:**

     $$
     \text{TE} \=\\sigma_{(r_p - r_{\rm ACWI})}\\sqrt{N}\\times100\%
     $$
   * **Interpretation:** Measures the consistency and magnitude of deviations from the benchmark.

9. **Information Ratio**

   * **Definition:** The ratio of average active return to tracking error.
   * **Formula:**

     $$
     \text{IR} \=\\frac{\overline{r_p - r_{\rm ACWI}}}{\sigma_{(r_p - r_{\rm ACWI})}}\\sqrt{N}
     $$
   * **Interpretation:** Assesses how efficiently the portfolio generates excess returns per unit of active risk.


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display, clear_output

# ───────────────────────────────────────────────────────────────
# 0)  SETUP: tickers, weights, download 5-yr daily closes
# ───────────────────────────────────────────────────────────────
tickers = df_combined['Ticker'].tolist() + ['ACWI']
prices  = yf.download(tickers, period='5y', interval='1d', auto_adjust=True,
                      progress=False)['Close']

# flatten MultiIndex if needed
if isinstance(prices.columns, pd.MultiIndex):
    prices = prices.droplevel(0, axis=1)

prices = prices.dropna(how='all').ffill()
rets = prices.pct_change().dropna()

# constant weights
w_port = (df_combined.set_index('Ticker')['Portfolio_Weight_%'] / 100)
w_port = w_port.reindex(prices.columns, fill_value=0)
w_port /= w_port.sum()

# return series
port_ret = rets.dot(w_port)
bm_ret   = rets['ACWI']

# ───────────────────────────────────────────────────────────────
# 1)  RISK METRIC FUNCTIONS
# ───────────────────────────────────────────────────────────────
TRADING_DAYS = 252
def volatility(r):        return r.std() * np.sqrt(TRADING_DAYS)
def beta(r, m):           return np.cov(r, m)[0,1] / m.var()
def tracking_error(r, m): return (r-m).std() * np.sqrt(TRADING_DAYS)
def info_ratio(r, m):     return (r.mean()-m.mean())*TRADING_DAYS / tracking_error(r, m)
def sharpe(r):            return r.mean()*TRADING_DAYS / volatility(r)
def sortino(r):
    down = r[r<0].std() * np.sqrt(TRADING_DAYS)
    return r.mean()*TRADING_DAYS / down if down!=0 else np.nan
def max_drawdown(r):
    cum  = (1+r).cumprod()
    peak = cum.cummax()
    return ((cum-peak)/peak).min()
def var_es(r, p=0.95):
    VaR = np.percentile(r, (1-p)*100)
    ES  = r[r<=VaR].mean()
    return VaR, ES

# ───────────────────────────────────────────────────────────────
# 2)  WINDOW DEFINITIONS
# ───────────────────────────────────────────────────────────────
windows = {
    '1m'  : 21,
    '3m'  : 63,
    '6m'  : 126,
    '12m' : 252,
 #   '2y'  : 504,
 #   '5y'  : 1260
}

# ───────────────────────────────────────────────────────────────
# 3)  METRIC TABLE BUILDER (Portfolio & Benchmark)
# ───────────────────────────────────────────────────────────────
def build_metrics(label):
    N = windows[label]
    pr = port_ret.tail(N)
    br = bm_ret.tail(N)

    # cumulative return over window
    cum_ret_p = (1+pr).prod() - 1
    cum_ret_b = (1+br).prod() - 1

    VaR_p, ES_p = var_es(pr)
    VaR_b, ES_b = var_es(br)

    data = {
        'Return %'         : [cum_ret_p*100, cum_ret_b*100],
        'Volatility %'     : [volatility(pr)*100, volatility(br)*100],
        'Sharpe Ratio'     : [sharpe(pr), sharpe(br)],
        'Sortino Ratio'    : [sortino(pr), sortino(br)],
        'Max Drawdown %'   : [max_drawdown(pr)*100, max_drawdown(br)*100],
        'VaR(95) %'        : [VaR_p*100, VaR_b*100],
        'ES(95) %'         : [ES_p*100,  ES_b*100],
        'Beta vs ACWI'     : [beta(pr, br), np.nan],
        'Tracking Error %' : [tracking_error(pr, br)*100, np.nan],
        'Info Ratio'       : [info_ratio(pr, br), np.nan]
    }

    idx = pd.Index(['Portfolio', 'Benchmark'], name='Series')
    return pd.DataFrame(data, index=idx).round(2)

# ───────────────────────────────────────────────────────────────
# 4)  ETF PERFORMANCE BUILDER  (unchanged)
# ───────────────────────────────────────────────────────────────
etf_tickers = df_combined.loc[df_combined['Sector']=='ETF holdings','Ticker'].unique().tolist()

def build_etf_metrics(label):
    if not etf_tickers:
        return pd.DataFrame()
    N = windows[label]
    rows = []
    for t in etf_tickers:
        r = rets[t].tail(N) # Fixed the mismatched parenthesis here
        cum_ret = (1+r).prod() - 1
        vol     = volatility(r)
        rows.append({
            'Ticker'              : t,
            'Cumulative Return %' : cum_ret*100,
            'Volatility %'        : vol*100
        })
    df_etf = pd.DataFrame(rows).set_index('Ticker').round(2)
    return df_etf

# ───────────────────────────────────────────────────────────────
# 5)  WIDGETS: Horizon Selector + Display
# ───────────────────────────────────────────────────────────────
win_dd = widgets.Dropdown(options=list(windows.keys()),
                          value='3m',
                          description='Window:')
out    = widgets.Output()

def _update_all(label):
    with out:
        clear_output(wait=True)
        print(f"### Risk Metrics (window: {label})")
        display(build_metrics(label))
        if etf_tickers:
            print("\n### ETF Performance Metrics")
            display(build_etf_metrics(label))

_update_all(win_dd.value)
win_dd.observe(lambda ch: _update_all(ch['new']), names='value')
display(win_dd, out)

Dropdown(description='Window:', index=1, options=('1m', '3m', '6m', '12m'), value='3m')

Output()

## Stress Testing

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm

ALPHA = 0.95                    # 95 % VaR
Z     = norm.ppf(1-ALPHA)       # ≈ –1.645

# ------------------------------------------------------------
# 1)  Marginal & Component VaR (variance–covariance method)
# ------------------------------------------------------------
def marginal_component_var(ret_matrix, weights, alpha=0.95):
    """
    Parameters
    ----------
    ret_matrix : DataFrame (dates × tickers), daily simple returns
    weights    : Series   (tickers), fractional weights summing to 1
    alpha      : float    VaR confidence level (e.g., 0.95)

    Returns
    -------
    df : DataFrame with columns
         ['Weight', 'Marginal_VaR', 'Component_VaR']
    """
    # 1)  Daily covariance matrix
    Sigma = ret_matrix.cov().values          # ndarray
    w     = weights.values.reshape(-1, 1)    # column vector

    port_std = np.sqrt(float(w.T @ Sigma @ w))          # σ_p
    port_VaR = -norm.ppf(1-alpha) * port_std            # positive number

    # 2)  Marginal VaR_i  = (Σ w)_i / σ_p  * z_{α}
    #     Component VaR_i = w_i * mVaR_i
    mVaR = (Sigma @ w).flatten() / port_std * (-norm.ppf(1-alpha))
    cVaR = weights.values * mVaR

    df = pd.DataFrame({
        'Weight'        : weights,
        'Marginal_VaR'  : mVaR,
        'Component_VaR' : cVaR
    })
    df.index.name = 'Ticker'
    df['% of Total VaR'] = df['Component_VaR'] / port_VaR * 100
    df.loc['TOTAL'] = [weights.sum(), np.nan, port_VaR, 100.0]

    return df.round(4)

# Example usage
var_table = marginal_component_var(rets, w_port, alpha=0.95)
display(var_table.head(15))

/tmp/ipython-input-266252091.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  port_std = np.sqrt(float(w.T @ Sigma @ w))          # σ_p


,Weight,Marginal_VaR,Component_VaR,% of Total VaR
Ticker,,,,
AAPL,0.0521,0.0252,0.0013,6.5317
ACWI,0.0000,0.0178,0.0000,0.0000
ADBE,0.0095,0.0193,0.0002,0.9097
AIQUY,0.0251,0.0097,0.0002,1.2063
AMD,0.0299,0.0419,0.0013,6.2409
AMZN,0.0541,0.0275,0.0015,7.4005
APD,0.0147,0.0162,0.0002,1.1909
BAC,0.0277,0.0198,0.0005,2.7271
COST,0.0314,0.0115,0.0004,1.8033


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd, numpy as np

# Define sector_map using the 'Ticker' and 'Sector' columns from df_combined
sector_map = df_combined.set_index('Ticker')['Sector']

# ───────────────────────────────────────────────────────────────
# 0)  Helper: stress function (same logic as before)
# ───────────────────────────────────────────────────────────────
def sector_stress_test(sector_shocks, weights, sector_map):
    tick_sector = sector_map.reindex(weights.index)
    shock_vec   = tick_sector.map(sector_shocks).fillna(0.0)
    pl_ticker   = weights * shock_vec
    total_pl    = pl_ticker.sum() * 100
    pl_sector   = (pl_ticker*100).groupby(tick_sector).sum().round(3)
    return total_pl, pl_sector.sort_values(ascending=False), (pl_ticker*100).round(3)

# ───────────────────────────────────────────────────────────────
# 1)  Build widgets for every sector
# ───────────────────────────────────────────────────────────────
unique_sectors = sorted(sector_map.unique())
slider_dict    = {
    sec: widgets.FloatSlider(
            value = 0.0, min = -0.20, max = 0.20, step = 0.01,
            description = sec[:14], readout_format = '.0%', layout = widgets.Layout(width='370px')
          )
    for sec in unique_sectors
}

ui_sliders = widgets.VBox(list(slider_dict.values()), layout=widgets.Layout(border='solid 1px #ddd', padding='5px'))
out        = widgets.Output()

# ───────────────────────────────────────────────────────────────
# 2)  Callback to recompute P/L whenever a slider moves
# ───────────────────────────────────────────────────────────────
def _refresh(*args):
    shocks = {sec: slider.value for sec, slider in slider_dict.items()}
    tot, by_sec, by_tkr = sector_stress_test(shocks, w_port, sector_map)
    with out:
        clear_output(wait=True)
        print(f"Total portfolio P/L under stress: {tot:.2f} %\n")
        display(by_sec.to_frame('Pct-pts').rename_axis('Sector'))
        print("\nTop 10 ticker contributions (pct-pts):")
        display(by_tkr.sort_values(ascending=False).head(10).to_frame('Pct-pts').rename_axis('Ticker'))

_refresh()  # initial
for slider in slider_dict.values():
    slider.observe(_refresh, names='value')

display(widgets.HBox([ui_sliders, out]))

## Combined Performance-and-Holdings Diagnostics

In [ ]:
import numpy as np, pandas as pd, ipywidgets as widgets
from IPython.display import display, clear_output

# ─────────────────────────────────────────────────────────────
# 0)  Helpers: sector maps & benchmark sector weights
# ─────────────────────────────────────────────────────────────
w_bench = (
    df_bm_sel.set_index('Ticker')['Benchmark_Weight_%']   # raw weights
      .groupby(level=0).sum()                             # collapse dups
      / 100                                               # convert % → fraction
)

# 1) Align to the return matrix columns (one column per unique ticker)
w_bench = w_bench.reindex(rets.columns, fill_value=0.0)

# ─────────────────────────────────────────────────────────────
# 1)  Function: sector-level Brinson-Fachler attribution
# ─────────────────────────────────────────────────────────────
def brinson_attrib(ret_df, w_port, w_bench, sector_map, window):
    """Return DF with allocation & selection (daily compounded)."""
    # slice last N rows
    ret = ret_df.tail(window)

    # sector returns
    port_sec_ret = (ret * w_port).groupby(sector_map, axis=1).sum().sum(axis=1)
    bench_sec_ret= (ret * w_bench).groupby(sector_map, axis=1).sum().sum(axis=1)

    r_port  = port_sec_ret.sum()
    r_bench = bench_sec_ret.sum()

    wp_sec = w_port.groupby(sector_map).sum()
    wb_sec = w_bench.groupby(sector_map).sum()

    # average weights over window (static weights assumed)
    alloc = (wp_sec - wb_sec) * bench_sec_ret.mean()
    selec = wb_sec * (port_sec_ret.mean() - bench_sec_ret.mean())
    inter = (wp_sec - wb_sec) * (port_sec_ret.mean() - bench_sec_ret.mean())

    df = pd.DataFrame({
        'Allocation %' : alloc*100,
        'Selection %'  : selec*100,
        'Interaction %': inter*100,
        'Total %'      : (alloc+selec+inter)*100
    }).round(3).sort_values('Total %', ascending=False)

    df.loc['__TOTAL__'] = df.sum()
    return df

# ─────────────────────────────────────────────────────────────
# 2)  Hit Ratio & Capture functions
# ─────────────────────────────────────────────────────────────
def hit_ratio(p, b):
    return (p > b).mean() * 100

def capture_ratios(p, b):
    up   = b[b>0]
    down = b[b<0]
    up_cap   = (p.loc[up.index].mean() / up.mean())   if len(up)>0   else np.nan
    down_cap = (p.loc[down.index].mean() / down.mean()) if len(down)>0 else np.nan
    return up_cap, down_cap

# ─────────────────────────────────────────────────────────────
# 3)  Window definitions & widgets
# ─────────────────────────────────────────────────────────────
windows = {'1m':21, '3m':63, '6m':126, '12m':252}
win_dd  = widgets.Dropdown(options=windows.keys(), value='3m', description='Window:')
out     = widgets.Output()

def _update(win_label):
    N = windows[win_label]
    pr = port_ret.tail(N)
    br = bm_ret.tail(N)

    with out:
        clear_output(wait=True)

        # ------ Attribution table ------
        print(f"## Brinson–Fachler Attribution ({win_label})")
        display(brinson_attrib(rets, w_port, w_bench, sector_map, N))

        # ------ Hit ratio & capture ------
        hr   = hit_ratio(pr, br)
        upc, dnc = capture_ratios(pr, br)
        print(f"\n## Hit / Capture ({win_label})")
        display(
            pd.DataFrame({
                'Metric' : ['Hit Ratio %', 'Upside Capture', 'Downside Capture'],
                'Value'  : [hr, upc, dnc]
            }).set_index('Metric').round(3)
        )

_update(win_dd.value)
win_dd.observe(lambda c: _update(c['new']), names='value')
display(win_dd, out)

Dropdown(description='Window:', index=1, options=('1m', '3m', '6m', '12m'), value='3m')

Output()